In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
%autosave 60

Autosaving every 60 seconds


# DSCI 235 Final Project
### By: Blake Davis and Elisabeth Ball

Data Source 1: https://www.kaggle.com/datasets/aliaamiri/2020-summer-olympics-dataset?select=2020_Olympics_Dataset.csv <br>
Data Source 2: https://www.kaggle.com/datasets/rio2016/olympic-games <br>
GitHub: https://github.com/bdavis12302/2020-2016-Summer-Olympic-Data <br>

## Project Description

For this project, we are analyzing data from the Tokyo 2020 Olympics and the Rio 2016 Olympics. The Olympic Games are a fascinating event, bringing the world together to appreciate excellence in athletics. The numerous different events, countries, and people competing in the Olympic Games give ample material to analyze. Finding patterns and interesting information in this Olympic data will likely change the way we see the games in the future and make us think about the nature of sports in a new way. This project will also be challenging and help us develop our teamwork, communication, and data wrangling skills. <br>

We are using 2 .csv files for our project. We have a main data set that is for the 2020 Summer Olympics in Tokyo. It includes data sorted by athlete name, with information on each athlete including age, country, sport, event, rank, and medal earned. The data set has multiple entries for each athlete as many athletes compete in multiple events. We have not decided how we are going to sort/filter the data for each individual athlete as they are each in the data set multiple times. We will use a 2016 Olympics in Rio De Janeiro data set to be able to determine if there were any repeat medalists and if there were more female or male medalists in 2016 vs. 2020. <br>

<hr>

We want to answer a few questions about the athletes that chose to compete in the 2016 and 2020 Summer Olympics:
1. What is the average age of an Olympic athlete in 2016? 2020? Compare.
   - Are there events that tend to have older compeditors? Meaning, is the mean age of athletes competing an a specific event greater than 30?<br>
6. Who are the 5 oldest athletes and who are the 5 youngest athletes in 2020?
   - What country are they from? <br>
   - What sport do they compete in? What is the age distribution of their sport in 2020? <br>
   - Did any of these athletes compete in 2016?
   - Do any of them compete in the events that tend to have older compeditors?
3. What are the top medaling countries in 2016? 2020? Compare. <br>
4. Does the United States have more female or male medalists in 2020? <br>
5. Which Olympic athlete has the most medals in 2016? 2020?
   - Is it the same person each Olympics? Same event? <br>

In [24]:
olympics2020 = pd.read_csv('https://raw.githubusercontent.com/bdavis12302/2020-2016-Summer-Olympic-Data/main/SummerOlympicDatasets/2020_Olympics_Dataset.csv', encoding='latin-1')
olympics2016 = pd.read_csv('https://raw.githubusercontent.com/bdavis12302/2020-2016-Summer-Olympic-Data/main/SummerOlympicDatasets/2016_athletes.csv', encoding='latin-1')
olympics2020

,Unnamed: 0,Code,Name,Gender,Age,NOC,Country,Discipline,Sport,Event,Rank,Medal
0,1,1346266,AALERUD Katrine,Female,26,NOR,Norway,CRD,Cycling Road,Women's Road Race,37.0,NaN
1,2,1346266,AALERUD Katrine,Female,26,NOR,Norway,CRD,Cycling Road,Women's Individual Time Trial,20.0,NaN
2,3,1355250,ABAD Nestor,Male,28,ESP,Spain,GAR,Artistic Gymnastics,Men's All-Around,NaN,NaN
3,4,1355250,ABAD Nestor,Male,28,ESP,Spain,GAR,Artistic Gymnastics,Men's Floor Exercise,NaN,NaN
4,5,1355250,ABAD Nestor,Male,28,ESP,Spain,GAR,Artistic Gymnastics,Men's Pommel Horse,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15116,15117,1317127,ZWOLINSKA Klaudia,Female,22,POL,Poland,CSL,Canoe Slalom,Women's Kayak,5.0,NaN
15117,15118,1428770,ZYKOVA Yulia,Female,25,ROC,Russia,SHO,Shooting,50m Rifle 3 Positions Women,2.0,Silver
15118,15119,1435606,ZYUZINA Ekaterina,Female,24,ROC,Russia,SAL,Sailing,Women's One Person Dinghy - Laser Radial,27.0,NaN
15119,15120,1325697,ZYZANSKA Sylwia,Female,24,POL,Poland,ARC,Archery,Women's Individual,33.0,NaN


## Part 1: What is the average age of an Olympic athlete in 2016? 2020? Compare

In [3]:
avgAge20 = olympics2020['Age'].mean()
avgAge20

26.765227167515377

In [25]:
olympics2016.loc[8546,'dob'] = "12/25/87"
startDate2016 = pd.to_datetime("08/05/16")
olympics2016['dob'] = pd.to_datetime(olympics2016['dob'])
olympics2016['age'] = abs(olympics2016['dob'] - startDate2016)
olympics2016['age'] = (np.floor((olympics2016['age'].dt.days)/365)).astype(int)
avgAge16 = olympics2016['age'].mean()
avgAge16

26.337233489339575